In [ ]:
import pandas as pd
import os
import csv
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.cm as cm
import seaborn as sns
import numpy as np
import glob

TEAMNAME = 'England'

#マスタデータのインポート
master = pd.read_csv(os.getcwd() + "/Resources/RugbyHub_master_data.csv",dtype=str)

df = pd.read_csv("/Users/isakakou/Desktop/Optaデータ関連/Six Nations BI CSV Files Apr 2025/941791_IRELvENGL_BI.csv",encoding='latin1')
df.columns
df[['playerName', 'teamName','x_coord','y_coord','x_coord_end','y_coord_end',
    'actionName','ActionTypeName','ActionResultName','qualifier3Name',
    'qualifier4Name','qualifier5Name','Metres','PlayNum',
    'SetNum','sequence_id','playerpositionName','refereeName','assistantreferee1Name','assistantreferee2Name']]
print(df.action)

In [ ]:
#チーム名指定でラインアウトデータ取得
df = df.loc[(df['teamName'] == TEAMNAME)]

dfaction = df.loc[df['actionName'] == 'Lineout Throw']
b = pd.crosstab([dfaction['qualifier3Name'],dfaction['ActionResultName']],dfaction['ActionTypeName'],margins=True)
b = b.reindex(columns=['Throw Front', 'Throw Middle', 'Throw Back', 'Throw 15m+'])

plt.gcf().subplots_adjust(wspace=4)
sns.heatmap(b,cmap='Blues', annot=True,annot_kws={'size':20})
plt.show()
plt.clf()

In [ ]:
dfaction = df.loc[(df['teamName'] == TEAMNAME)]

dfaction = dfaction.loc[dfaction['actionName'] == 'Ruck']
rspeed = ['0-1 Seconds','1-2 Seconds','3-4 Seconds','4-5 Seconds','5-6 Seconds','6+ Seconds', "N/A Ruck Speed"]
cl = ['y','m', 'c', 'r', 'g', 'b', (0,0.3,0.5), (0,0,0)]
fig = plt.figure(figsize=(7, 10))
ax = fig.add_subplot(1, 1,1)
plt.xlim(0, 68)
plt.ylim(0,100)

for i in range(len(rspeed)):
    r = dfaction.loc[dfaction['qualifier4Name'] == rspeed[i]]
    plt.scatter(r['y_coord'], r["x_coord"], label=rspeed[i] ,marker = 'D',color=cl[i], s=100)

#背景画像の設定
fig.patch.set_facecolor('white')
im = Image.open(os.getcwd()+"/Resources/FIELD_image.jpeg")
xlim = ax.get_xlim()
ylim = ax.get_ylim()
plt.imshow(im, extent=[*xlim, *ylim], aspect='auto', alpha=0.6)

#ラベルの表示
plt.title(TEAMNAME)
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize = 12)
plt.gcf().subplots_adjust(wspace=4)


In [ ]:
#Kickが入力された場合の処理-----
dfaction = df.loc[(df['teamName'] == TEAMNAME)]

dfaction = dfaction.loc[dfaction['actionName'] == 'Kick']
kicktypes = ['Bomb', 'Chip', 'Cross Pitch', 'Territorial', 'Low', 'Box', 'Touch Kick','Kick Error']
kicks = dfaction.loc[dfaction['qualifier3'] != 148]

cl = ['y','m', 'c', 'r', 'g', 'b', (0,0.3,0.5), (0,0,0)]
fig = plt.figure(figsize=(7, 10))
ax = fig.add_subplot(1, 1,1)
plt.xlim(0, 70)
plt.ylim(0,100)

print(kicks[['playerName','MatchTime','ActionTypeName']])
for i in range(len(kicktypes)):
    plts = kicks.loc[kicks['ActionTypeName']== kicktypes[i]]
    plt.quiver(plts['y_coord'], plts["x_coord"], (plts['y_coord_end']-plts['y_coord']), (plts["x_coord_end"]-plts['x_coord']), angles="xy", scale_units='xy',label= kicktypes[i], color = cl[i], scale=1,width=0.008, headwidth=5, headlength = 8, headaxislength = 7, pivot='tail')
#Kickの処理ここまで------

#背景画像の設定
fig.patch.set_facecolor('white')
im = Image.open(os.getcwd()+"/Resources/FIELD_image.jpeg")
xlim = ax.get_xlim()
ylim = ax.get_ylim()
plt.imshow(im, extent=[*xlim, *ylim], aspect='auto', alpha=0.6)

#ラベルの表示
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize = 12)
plt.title(TEAMNAME)
plt.gcf().subplots_adjust(wspace=4)

#表を出力
player_kicks = pd.crosstab([kicks['playerName'],kicks['ActionTypeName']],kicks['ActionResultName'])
player_kicks

In [ ]:
dfaction = df.loc[(df['teamName'] == TEAMNAME)]
dfaction = dfaction.loc[dfaction['actionName'] == 'Tackle']
dfaction[['playerName','actionName','ActionResultName','ActionTypeName','qualifier3Name','qualifier4Name','qualifier5Name']]

In [ ]:
#TODO エリアごとにラインアウトを整理する
df = df.loc[(df['teamName'] == TEAMNAME)]

dfaction = df.loc[df['actionName'] == 'Lineout Throw']

areas = ['0-22','22-50','50-22','22-100']
Metres_for_area = [0,22,50,73,100]

dfaction['x_coord_category'] = np.nan

for i in range(len(areas)):
    print(i)
    print(areas[i])
    dfaction.x_coord_category[(dfaction['x_coord'].astype(int) < Metres_for_area[i+1]) & (dfaction['x_coord'].astype(int) >= Metres_for_area[i])] = areas[i]

lineout_each_area = pd.crosstab(dfaction['x_coord_category'],dfaction['ActionTypeName'],margins=True).reindex(columns=['Throw Front', 'Throw Middle', 'Throw Back', 'Throw 15m+'],index=list(reversed(areas))).fillna(0)
lineout_each_area
lineout_member_each_area = pd.crosstab(dfaction['x_coord_category'],dfaction['Metres']).reindex(index=list(reversed(areas))).fillna(0)

plt.gcf().subplots_adjust(wspace=4)
sns.heatmap(lineout_each_area,cmap='Blues', annot=True,annot_kws={'size':20})
plt.show()
plt.clf()

plt.gcf().subplots_adjust(wspace=4)
sns.heatmap(lineout_member_each_area,cmap='Blues', annot=True,annot_kws={'size':20})
plt.show()
plt.clf()

In [ ]:
import pandas as pd
import os
import csv
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.cm as cm
import seaborn as sns
import numpy as np
import glob

TEAMNAME = 'Ireland'

# パスで指定したファイルの一覧をリスト形式で取得.
csv_files = glob.glob('/Users/isakakou/Desktop/Optaデータ関連/Six Nations BI CSV Files Apr 2025/*.csv')

#読み込むファイルのリストを表示
for a in csv_files:
    print(a)

#csvファイルの中身を追加していくリストを用意
data_list = []

#読み込むファイルのリストを走査
for file in csv_files:
    data_list.append(pd.read_csv(file,encoding='latin1'))
    print(file)

#リストを全て行方向に結合
#axis=0:行方向に結合, sort
df = pd.concat(data_list, axis=0, sort=True)

# df.to_csv("/Users/isakakou/Desktop/22-23_Stats/total.csv",index=False)

#Kickが入力された場合の処理-----
dfaction = df.loc[(df['teamName'] == TEAMNAME)]

dfaction = dfaction.loc[dfaction['actionName'] == 'Kick']
kicktypes = ['Bomb', 'Chip', 'Cross Pitch', 'Territorial', 'Low', 'Box', 'Touch Kick','Kick Error']
kicks = dfaction.loc[dfaction['qualifier3'] != 148]

cl = ['y','m', 'c', 'r', 'g', 'b', (0,0.3,0.5), (0,0,0)]
players = kicks.drop_duplicates(subset=['playerName'],keep='first')
player_list = players['playerName'].reset_index(drop=True)
print(player_list)

for p in range(len(player_list)):
    fig = plt.figure(figsize=(7, 10))
    ax = fig.add_subplot(1, 1,1)
    plt.xlim(0, 68)
    plt.ylim(0,100)
    for i in range(len(kicktypes)):
        plts = kicks.loc[(kicks['ActionTypeName']== kicktypes[i])&(kicks['playerName'] == player_list[p])]
        plt.quiver(plts['y_coord'], plts["x_coord"], (plts['y_coord_end']-plts['y_coord']), (plts["x_coord_end"]-plts['x_coord']), angles="xy", scale_units='xy',label= kicktypes[i], color = cl[i], scale=1,width=0.008, headwidth=5, headlength = 8, headaxislength = 7, pivot='tail')
    #Kickの処理ここまで------

    #背景画像の設定
    fig.patch.set_facecolor('white')
    im = Image.open(os.getcwd()+"/Resources/FIELD_image.jpeg")
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    plt.imshow(im, extent=[*xlim, *ylim], aspect='auto', alpha=0.6)
    position = dfaction.loc[dfaction['playerName'] == player_list[p]]['playerpositionName'].drop_duplicates(keep='first').reset_index(drop=True)

    #ラベルの表示
    lgd = plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize = 12)
    plt.title(player_list[p] + "\n Position: " + position[0])
    plt.gcf().subplots_adjust(wspace=4)
    plt.show()
    
    
# 接点無双度計測
def at_settenmusou(TEAMNAME):
    dfaction = df.loc[(df['teamName'] == TEAMNAME)]
    carries = dfaction.loc[dfaction['actionName'] == 'Carry']
    musou = carries.loc[(carries["qualifier4Name"] == 'Dominant Contact') | (carries["qualifier4Name"] == 'Neutral Contact')]
    print(f"{TEAMNAME} AT接点無双度: {len(musou)/len(carries)*100}%")
    
def df_settenmusou(TEAMNAME):
    team_df = df.loc[(df['homeTeamName'] == TEAMNAME) | (df['awayTeamName'] == TEAMNAME)]
    dfaction = team_df.loc[(team_df['teamName'] != TEAMNAME)]
    carries = dfaction.loc[dfaction['actionName'] == 'Carry']
    musou = carries.loc[(carries["qualifier3Name"] == 'Neutral Gain line') | (carries["qualifier3Name"] == 'Failed Gain line')]
    print(f"{TEAMNAME} DF接点無双度: {len(musou)/len(carries)*100}%")
    
#SIX_NATIONS_TEAMS = ["France", "Ireland", "England", "Scotland", "Wales", "Italy"]
#for team in SIX_NATIONS_TEAMS:
    #at_settenmusou(team)
    #df_settenmusou(team)
    
def calc_possession_start_end_probability(df):
    # Possession アクションだけを抽出
    df_possession = df[df["actionName"] == "Possession"]

    # 開始と終了のペアを作成
    pair_counts = df_possession.groupby(["ActionTypeName", "ActionResultName"]).size().reset_index(name="count")

    # 総数で割って確率に変換
    total_possessions = pair_counts["count"].sum()
    pair_counts["probability"] = (pair_counts["count"] / total_possessions)

    # 結果をソートして見やすく
    pair_counts = pair_counts.sort_values(by="probability", ascending=False).reset_index(drop=True)

    return pair_counts

def calc_possession_probability_crosstab(df):
    # Possession のみに絞る
    df_possession = df[df["actionName"] == "Possession"]

    # 各 (ActionTypeName, ActionResultName) の出現回数をカウント
    crosstab_count = pd.crosstab(df_possession["ActionTypeName"], df_possession["ActionResultName"])

    # 各行（ActionTypeName）ごとの合計で割って、確率を計算
    crosstab_prob = crosstab_count.div(crosstab_count.sum(axis=1), axis=0)

    return crosstab_prob

pair_counts = calc_possession_start_end_probability(df)
display(pair_counts)
# 各ポゼッション起点がトライに繋がる確率
print(pair_counts.loc[pair_counts["ActionResultName"] == "Try"])
# クロス集計表として表示
display(calc_possession_probability_crosstab(df))

def calc_possession_value_model():
    #FXIDが同じでPLIDがチームのものについて、特定のアクションから特定のアクションに移行する確率を計算する
    print("いつかやる")